- First start with Jane Austen books tokenized. 
- Using original word at this point.
- N-gram window needs to through one column across n-rows.

In [2]:
import os, sys, re, json, time, csv, copy, random, unittest
import itertools, collections

import numpy as np
from scipy import stats

import nltk

from shared_lib import utils, vocabulary, ngram_lm, ngram_utils

random.seed(10)

In [7]:
austen_emma_all = []
with open('../book-nlp-master/data/tokens/austen.emma.tokens', 'rb') as f:
    reader = csv.reader(f, dialect="excel-tab")
    austen_emma_all = list(reader)
# Remove header
austen_emma_all = austen_emma_all[1:]

In [8]:
austen_emma_tokens = [line[7] for line in austen_emma_all]
print 'The first 10 tokens from Jane Austen\'s Emma:'
print ', '.join(austen_emma_tokens[0:10])

The first 10 tokens from Jane Austen's Emma:
Produced, by, An, Anonymous, Volunteer, EMMA, By, Jane, Austen, VOLUME


In [9]:
# Shuffle when needed
random.shuffle(austen_emma_tokens)

In [10]:
tokens_length = len(austen_emma_tokens)
print tokens_length * 0.8
V = 30000
train_tokens  = austen_emma_tokens[ : int(tokens_length * 0.8)]
test_tokens = austen_emma_tokens[int(tokens_length * 0.8) : ]
vocab = vocabulary.Vocabulary((utils.canonicalize_word(w) for w in train_tokens), size=V)
print "Train set vocabulary: %d words" % vocab.size
print "Train set tokens: %d " % len(train_tokens)
print "Test set tokens: %d " % len(test_tokens)
print "First 10 Train Tokens: ", train_tokens[0:10]
print "First 10 Test Tokens: ", test_tokens[0:10]

154540.0
Train set vocabulary: 6704 words
Train set tokens: 154540 
Test set tokens: 38635 
First 10 Train Tokens:  [',', ',', 'greater', 'the', 'that', 'obliged', 'with', 'properly', 'spot', 'obliging']
First 10 Test Tokens:  ['visit', 'dinner', 'silence', 'Miss', 'the', 'my', 'the', 'higher', 'nothing', 'to']


In [11]:
'''
Feeding original text instead of the tokens generated to see if this works immly with the code from assignment-2
'''
from nltk.corpus import gutenberg
assert(nltk.download('gutenberg'))
V = 10000
corpus = nltk.corpus.gutenberg
train_sents, test_sents = utils.get_train_test_sents(corpus, split=0.8, shuffle=False)
vocab = vocabulary.Vocabulary((utils.canonicalize_word(w) for w in utils.flatten(train_sents)), size=V)
# vocab = vocabulary.Vocabulary((utils.canonicalize_word(w) for w in utils.flatten(corpus.sents())), size=V)
print "Train set vocabulary: %d words" % vocab.size

[nltk_data] Downloading package gutenberg to
[nltk_data]     /home/sharmila_velamur/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


TypeError: len() of unsized object

In [5]:

def sents_to_tokens(sents):
    """Returns an flattened list of the words in the sentences, with padding for a trigram model."""
    padded_sentences = (["<s>", "<s>"] + s + ["</s>"] for s in sents)
    # This will canonicalize words, and replace anything not in vocab with <unk>
    return np.array([utils.canonicalize_word(w, wordset=vocab.wordset) 
                     for w in utils.flatten(padded_sentences)], dtype=object)

train_tokens = sents_to_tokens(train_sents)
test_tokens = sents_to_tokens(test_sents)
print "Sample data: \n" + repr(train_tokens[:20])

NameError: name 'train_sents' is not defined

In [25]:
Model = ngram_lm.KNTrigramLM
t0 = time.time()
print "Building trigram LM...",
lm = Model(train_tokens)
print "done in %.02f s" % (time.time() - t0)
ngram_utils.print_stats(lm)

Building trigram LM... done in 10.84 s
=== N-gram Language Model stats ===
10000 unique 1-grams
319307 unique 2-grams
968394 unique 3-grams
Optimal memory usage (counts only): 27 MB


In [28]:
max_length = 20
num_sentences = 5

for _ in range(num_sentences):
    seq = ["<s>", "<s>"]
    for i in range(max_length):
        seq.append(ngram_utils.predict_next(lm, seq))
        # Stop at end-of-sentence.
        if seq[-1] == "</s>": break
    print " ".join(seq)
    print "[{1:d} tokens; log P(seq): {0:.02f}]".format(*ngram_utils.score_seq(lm, seq))
    print ""

<s> <s> but by her husband , and exalt the baby ' s a certain man of god . </s>
[17 tokens; log P(seq): -84.42]

<s> <s> he was a long time , at last , in . </s>
[11 tokens; log P(seq): -53.02]

<s> <s> and the five hundred a - rum smell asked him , the family of the holy things of this ,
[20 tokens; log P(seq): -94.37]

<s> <s> DGDG : DGDG and again from the lord ; they hear not : i am not at liberty to think
[20 tokens; log P(seq): -79.48]

<s> <s> " then the true english your honour and majesty ," said turnbull , with a long , green herb have
[20 tokens; log P(seq): -106.30]



In [14]:
lm.set_live_params(k = 0.001, delta=0.75)

In [26]:
log_p_data, num_real_tokens = ngram_utils.score_seq(lm, train_tokens)
print "Train perplexity: %.02f" % (2**(-1*log_p_data/num_real_tokens))

Train perplexity: 19.55


In [27]:
log_p_data, num_real_tokens = ngram_utils.score_seq(lm, test_tokens)
print "Test perplexity: %.02f" % (2**(-1*log_p_data/num_real_tokens))

Test perplexity: 302.38
